# Sentiment Analysis
## First project of Pattern Recognition Course

Professor : Dr. Analouyi

TAs : Mr.Pourbagheri and Mr. Tabatabaee

Maryam Sadat Hashemi
94523252

Add the prerequisites.

In [85]:
import numpy as np
import pandas as pd
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer
import heapq 
import numpy as np
import sklearn

# Step 1 :  Download and Combine Movie Reviews
First of All, we download IMDB reviews from <a href="https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz">this</a>. Next, we merge all postive and negative files of train and test into train_pos.txt, train_neg.txt, test_pos.txt and test_neg.txt respectively. Then we save all these 4 file in a directory called movie_data.

FOR %f IN (*.txt) DO type %f >> train_neg.txt & echo. >> train_neg.txt

# Step 2: Read into Python
For most of what we want to do in this walkthrough, we’ll only need our reviews to be in a Python list.

In [54]:
reviews_train_pos = []
for line in open('movie_data/train_pos.txt', 'r', encoding='utf-8'):
    reviews_train_pos.append(line)
    
reviews_train_neg = []
for line in open('movie_data/train_neg.txt', 'r',encoding='utf-8'):
    reviews_train_neg.append(line)
    
reviews_test_pos = []
for line in open('movie_data/test_pos.txt', 'r',encoding='utf-8'):
    reviews_test_pos.append(line)
    
reviews_test_neg = []
for line in open('movie_data/test_pos.txt', 'r',encoding='utf-8'):
    reviews_test_neg.append(line)

full_train = []
for line in open('movie_data/full_train.txt', 'r',encoding='utf-8'):
    full_train.append(line)
    
full_test = []
for line in open('movie_data/full_test.txt', 'r',encoding='utf-8'):
    full_test.append(line)
    
print(len(reviews_train_pos))
print(len(reviews_train_neg))
print(len(reviews_test_pos))
print(len(reviews_test_neg))
print(len(full_train))
print(len(full_test))


25001
25001
25001
25001
50002
50002


# Step 3: Clean and Preprocess

The raw text is pretty messy. So before we can do any analytics, we need to clean things up. Here’s one example:

In [55]:
print(reviews_test_neg[1000])

This is halfway to being a top movie. The opening section, which spoofs Hollywood "social message" films is absolutely brilliant. It is a riot from start to finish.<br /><br />The second section, which introduces us to the main characters of the story is really great too. We get a lot of great comic setups, top notch performances, and the dialog is really dynamic.<br /><br />(Spoiler warning!)<br /><br />The one think that really annoyed me about this film though is the ending, which I think contradicts everything that went before. My interpretation was that this film was taking the mickey out all the silly prejudices and innuendo of small town gossip and national tabloid sensationalism. I loved that the film was championing the cause that a person's sexuality is NOT determined by their hobbies, idiosyncrasies, fashion sense or whatever. And then the ending goes and re-enforces all the gossip and stereotypes that the movie successfully lampooned in the first place. It turns out everyon

So, we remove these characters \ ; : ! ' ? " ( ) \[ \] , from review and we also replace these characters -, / and HTML tag with space.

In [56]:
REPLACE_NO_SPACE = re.compile("[\\;:!\',\"()\[\]&%*@#$0-9]")
REPLACE_WITH_SPACE = re.compile("(<.*?>)|<!--.*?-->|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

reviews_train_pos = preprocess_reviews(reviews_train_pos)
reviews_train_neg = preprocess_reviews(reviews_train_neg)
reviews_test_pos = preprocess_reviews(reviews_test_pos)
reviews_test_neg = preprocess_reviews(reviews_test_neg)
full_train = preprocess_reviews(full_train)
full_test = preprocess_reviews(full_test)

print(len(reviews_train_pos))
print(len(reviews_train_neg))
print(len(reviews_test_pos))
print(len(reviews_test_neg))
print(len(full_train))
print(len(full_test))

25001
25001
25001
25001
50002
50002


And this is what the same review looks like now:

In [57]:
print(reviews_test_neg[1000])

this is halfway to being a top movie. the opening section which spoofs hollywood social message films is absolutely brilliant. it is a riot from start to finish.  the second section which introduces us to the main characters of the story is really great too. we get a lot of great comic setups top notch performances and the dialog is really dynamic.  spoiler warning  the one think that really annoyed me about this film though is the ending which i think contradicts everything that went before. my interpretation was that this film was taking the mickey out all the silly prejudices and innuendo of small town gossip and national tabloid sensationalism. i loved that the film was championing the cause that a persons sexuality is not determined by their hobbies idiosyncrasies fashion sense or whatever. and then the ending goes and re enforces all the gossip and stereotypes that the movie successfully lampooned in the first place. it turns out everyone was  right godamit this was very disappoi

# Step 4: Vectorization and Feature Extraction
In order for this data to make sense to our machine learning algorithm, we’ll need to convert each review to a numeric representation, which we call vectorization.

## Bag of Words
Bag of Words (BOW) is a method to extract features from text documents. These features can be used for training machine learning algorithms. It creates a vocabulary of all the unique words occurring in all the documents in the training set. 

In simple terms, it’s a collection of words to represent a sentence with word count and mostly disregarding the order in which they appear. 

### Step 1: Obtaining most frequent words
we obtain most frequent words in our training set.We declare a dictionary to hold our bag of words. Next we tokenize each review to words and we apply lemmatizing and stemming in each word. Now for each word, we check if the word exists in our dictionary. If it does, then we increment its count by 1. If it doesn’t, we add it to our dictionary and set its count as 1.
We do not need to use all those words. Hence, we select a particular number of most frequently used words(count(word)>5).

In [63]:
def frequent_words(dataset):
    lemmatizer = WordNetLemmatizer()
    ps = PorterStemmer() 

    # Creating the Bag of Words model 
    word2count = {} 
    for data in dataset: 
        words = nltk.word_tokenize(data) 
        for word in words: 
            word = lemmatizer.lemmatize(word)
            word = ps.stem(word)
            if word not in word2count.keys(): 
                word2count[word] = 1
            else: 
                word2count[word] += 1

    
    freq_words = {k: v for k, v in word2count.items() if v >= 5}
    print("count word2count = " + str(len(word2count)))
    print("count freq_words = " + str(len(freq_words)))
    
    return freq_words

In [64]:
freq_words_train = frequent_words(full_train)

count word2count = 62669
count freq_words = 25671


### Step 2: Building the Bag of Words model
In this step we construct a vector for each review from frequent words.

In [80]:
def BOW(dataset, freq_words):
    lemmatizer = WordNetLemmatizer()
    ps = PorterStemmer() 
    
    X = [] 
    sample = {k: 0 for k, v in freq_words.items()}
    print(len(sample))
    
    for data in dataset: 
        vector = sample
        words = nltk.word_tokenize(data)
        for word in words: 
            word = lemmatizer.lemmatize(word)
            word = ps.stem(word)
            if word in vector.keys(): 
                vector[word] += 1  
        X.append(vector) 
    
    print(len(X))
    X = np.asarray(X)
    return X

In [81]:
X_train = BOW(full_train,freq_words_train)

25671
50002


In [83]:
print(len(X_train))
print(X_train[1000])

50002
{'stori': 26251, 'of': 291678, 'a': 418282, 'man': 11977, 'who': 42359, 'ha': 33853, 'unnatur': 109, 'feel': 10589, 'for': 88539, 'pig': 291, '.': 546895, 'start': 8321, 'out': 34307, 'with': 88115, 'open': 4195, 'scene': 21129, 'that': 146824, 'is': 214337, 'terrif': 899, 'exampl': 3089, 'absurd': 807, 'comedi': 7330, 'formal': 73, 'orchestra': 97, 'audienc': 5329, 'turn': 7757, 'into': 18207, 'an': 43083, 'insan': 675, 'violent': 1116, 'mob': 328, 'by': 45028, 'the': 671188, 'crazi': 1388, 'chant': 84, 'it': 207325, 'singer': 700, 'unfortun': 3100, 'stay': 2514, 'whole': 6156, 'time': 32181, 'no': 25302, 'gener': 4030, 'narr': 898, 'eventu': 1554, 'make': 30374, 'just': 35452, 'too': 15580, 'off': 12172, 'put': 6258, 'even': 25710, 'those': 9378, 'from': 40940, 'era': 1272, 'should': 9916, 'be': 67356, 'cryptic': 40, 'dialogu': 3282, 'would': 24460, 'shakespear': 594, 'seem': 14426, 'easi': 1602, 'to': 271319, 'third': 1546, 'grader': 76, 'on': 68276, 'technic': 1026, 'level': 

## BERT Embedding

### 1.vector for each sentences

### 2. vector for each word

## TF_IDF with stop words(optional)

## TF_IDF without stop words(optional)

## Word2Vec with stop words(optional)

## Word2Vec without stop words(optional)

# Step 5: Build Classifier
Now that we’ve transformed our dataset into a format suitable for modeling, we can start building a classifier.

## Naive Bayes

In [ ]:
def NaiveBayes(X_train, Y_train, X_test, Y_test):
    # alpha = smoothing parameter (0 for no smoothing)
    # fit_prior = Only used in edge case with a single class in the training set.
    # class_prior = Prior probabilities of the classes. Not used.
    # norm = Whether or not a second normalization of the weights is performed.
    # ComplementNB(alpha=1.0, fit_prior=True, class_prior=None, norm=False).
    clf = sklearn.naive_bayes.ComplementNB(alpha=1.0, class_prior=None, fit_prior=True, norm=False)
    clf.fit(X_train, Y_test)
    Y_predict = predict(X_test)
    return Y_predict
    

## SVM

In [ ]:
def SVM(X_train, Y_train, X_test, Y_test):
    #sklearn.svm.SVC(C=1.0, kernel=’rbf’, degree=3, gamma=’auto_deprecated’, 
    #coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, 
    #class_weight=None, verbose=False, max_iter=-1, decision_function_shape=’ovr’, 
    #random_state=None)
    clf = sklearn.svm.SVC(gamma='auto')
    clf.fit(X_train, Y_train) 
    Y_predict = clf.predict(X_test)
    return Y_predict


## Decision Tree

In [ ]:
def decision_tree(X_train, Y_train, X_test, Y_test):
    # sklearn.tree.DecisionTreeClassifier(criterion=’gini’, splitter=’best’,
    # max_depth=None, min_samples_split=2, min_samples_leaf=1, 
    # min_weight_fraction_leaf=0.0, max_features=None, random_state=None, 
    # max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
    # class_weight=None, presort=False)
    clf = sklearn.tree.DecisionTreeClassifier()
    clf = clf.fit(X_train, Y_train)
    Y_predict = clf.predict(X_test)
    return Y_predict
    

## Random Forest

In [ ]:
def random_forest(X_train, Y_train, X_test, Y_test):
    # sklearn.ensemble.RandomForestClassifier(n_estimators=’warn’, 
    # criterion=’gini’, max_depth=None, min_samples_split=2, 
    # min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, 
    # max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None,
    # bootstrap=True, oob_score=False, n_jobs=None, random_state=None, 
    # verbose=0, warm_start=False, class_weight=None)
    clf = sklearn.ensemble.RandomForestClassifier()
    clf = clf.fit(X_train, Y_train)
    Y_predict = clf.predict(X_test) 
    return Y_predict

# Step 6: Results

### Confusion Matrix

### ROC

### AUC

### Precision

### Recall

### 